In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

2.0.0


In [2]:
training_set = pd.read_csv("train.csv")
training_imgs = [ x for x in list(training_set.Image)]

In [3]:
print(training_imgs)

['image7042.jpg', 'image3327.jpg', 'image10335.jpg', 'image8019.jpg', 'image2128.jpg', 'image1106.jpg', 'image6750.jpg', 'image5508.jpg', 'image9571.jpg', 'image9613.jpg', 'image1719.jpg', 'image61.jpg', 'image10285.jpg', 'image129.jpg', 'image6760.jpg', 'image192.jpg', 'image2008.jpg', 'image3892.jpg', 'image9817.jpg', 'image3123.jpg', 'image5544.jpg', 'image1854.jpg', 'image9015.jpg', 'image5318.jpg', 'image6328.jpg', 'image6553.jpg', 'image8453.jpg', 'image10333.jpg', 'image8262.jpg', 'image8146.jpg', 'image9397.jpg', 'image2808.jpg', 'image9384.jpg', 'image3467.jpg', 'image7522.jpg', 'image5373.jpg', 'image3640.jpg', 'image9556.jpg', 'image6311.jpg', 'image1265.jpg', 'image4189.jpg', 'image5948.jpg', 'image9184.jpg', 'image3049.jpg', 'image299.jpg', 'image748.jpg', 'image7626.jpg', 'image137.jpg', 'image6276.jpg', 'image915.jpg', 'image3861.jpg', 'image6047.jpg', 'image9657.jpg', 'image6031.jpg', 'image6413.jpg', 'image7253.jpg', 'image7676.jpg', 'image9960.jpg', 'image4321.jpg', '

In [4]:
training_set.Class = pd.factorize(training_set.Class)[0]
training_set.Class = training_set.Class.astype(str)
print(training_set)

               Image Class
0      image7042.jpg     0
1      image3327.jpg     1
2     image10335.jpg     2
3      image8019.jpg     0
4      image2128.jpg     2
...              ...   ...
5978   image2494.jpg     0
5979   image8015.jpg     2
5980   image4975.jpg     0
5981   image1222.jpg     0
5982   image9635.jpg     2

[5983 rows x 2 columns]


In [5]:
training_labels = list(training_set['Class'])
training_set = pd.DataFrame( {'Image': training_imgs,'Class': training_labels})

In [6]:
print(training_set.head(5))

            Image Class
0   image7042.jpg     0
1   image3327.jpg     1
2  image10335.jpg     2
3   image8019.jpg     0
4   image2128.jpg     2


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

In [9]:
train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="Train Images",x_col="Image",
                                        y_col="Class",
                                        class_mode="categorical",
                                        target_size=(128,128),
                                        batch_size=50)

Found 5983 validated image filenames belonging to 4 classes.


In [56]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape= (128, 128, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(4, activation='softmax'))

In [58]:
# compile model
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['categorical_accuracy','accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 128)      

In [59]:
model.fit_generator(train_generator, epochs = 100, steps_per_epoch = 50)

In [60]:
test_set = pd.read_csv("test.csv")
test_imgs = [x for x in list(test_set.Image)]
test_set = pd.DataFrame( {'Image': test_imgs })

In [19]:
from keras.preprocessing import image

Y_pred = []

for i in range(len(test_set)):
    img = image.load_img(path="Test Images/" + test_set.Image[i] ,target_size=(128,128,3))
    img = image.img_to_array(img)
    img = img/255.0
    test_img = img.reshape((1,128,128,3))
    img_class = model.predict_classes(test_img)
    prediction = img_class[0]
    Y_pred.append(prediction)

In [20]:
predicted = []
for i in range(0, len(Y_pred)):
    if (Y_pred[i] == 0):
        predicted.append("Food")
    elif (Y_pred[i] == 1):
        predicted.append("misc")
    elif (Y_pred[i] == 2):
        predicted.append("Attire")
    else:
        predicted.append("Decorationandsignage")

In [21]:
print(predicted)

['Food', 'Food', 'Food', 'Food', 'Attire', 'Food', 'misc', 'misc', 'misc', 'Food', 'Decorationandsignage', 'Decorationandsignage', 'Food', 'Attire', 'Food', 'Attire', 'Attire', 'Food', 'misc', 'Food', 'Food', 'Attire', 'Food', 'Attire', 'Food', 'Attire', 'Food', 'Attire', 'Food', 'Food', 'Attire', 'Attire', 'misc', 'Attire', 'Decorationandsignage', 'Food', 'Attire', 'Attire', 'Food', 'misc', 'Attire', 'Attire', 'Attire', 'Attire', 'misc', 'Food', 'misc', 'Food', 'Food', 'Food', 'Food', 'Attire', 'misc', 'misc', 'Food', 'Food', 'Food', 'Attire', 'Decorationandsignage', 'Attire', 'misc', 'Decorationandsignage', 'Decorationandsignage', 'Food', 'Food', 'Food', 'Attire', 'Attire', 'Attire', 'misc', 'Attire', 'Food', 'Decorationandsignage', 'Attire', 'Attire', 'Food', 'Attire', 'Attire', 'Food', 'misc', 'Food', 'Food', 'Food', 'Food', 'misc', 'Food', 'Food', 'Attire', 'Attire', 'Decorationandsignage', 'Food', 'Food', 'Attire', 'Attire', 'Food', 'Food', 'Attire', 'Attire', 'Food', 'misc', 'mi

In [55]:
pd.DataFrame(list(zip(test_set.Image, predicted)),columns=["Image","Class"]).to_csv("submission.csv",index=None)